In [ ]:
print("\n".join(list([el.replace(", ", ",").replace("/ ", "/").replace(" ", "_") for el in set(hist.keys()) if "WT" not in el])))

In [2]:
import pandas as pd

data = pd.read_json(path_or_buf="datasets/all.jsonl", lines=True)

In [3]:
data

,id,text,Comments,label
0,26,От главы ВОЗ Гебреисуса потребовали прекратить...,[],"[[0, 68, (WT) Opinion], [192, 320, Appeal to a..."
1,28,Начало. Коронавирус COVID-19 пришелец из космо...,[],"[[0, 49, (WT) Opinion], [0, 49, (WTO) Irreleva..."
2,29,"Загадка коронавируса, чипирование и новое буду...",[],"[[0, 49, (WT) Opinion], [0, 49, (WTO) Obfuscat..."
3,30,От вакцинации от коронавируса к чипированию //...,[],"[[0, 43, (WT) Opinion], [0, 43, (WTO) Irreleva..."
4,31,Наталья Попова’ (2020) КОРОНАВИРУСА НЕ СУЩЕСТВ...,[],"[[23, 50, (WTO) Irrelevant data / uncheckable ..."
...,...,...,...,...
120,22,Зачем Билл Гейтс хочет нас чипировать под пред...,[],"[[0, 66, (WT) Opinion], [189, 672, Casting Dou..."
121,23,"Прикрываясь коронавирусом, в США активно продв...",[],"[[0, 78, (WT) Opinion], [0, 78, (WTO) Irreleva..."
122,24,Антиоксиданты против Covid // Яндекс. Дзен. На...,[],"[[0, 26, (WT) Opinion], [0, 26, (WTO) Irreleva..."
123,25,Преемник преподобного Паисия старец Евфимий о ...,[],"[[0, 108, (WT) Other], [273, 734, Appeal to au..."


In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=57)

In [29]:
import os

def create_set(data, text_folder, labels_file, save_labels=True):
    if os.path.exists(labels_file):
        os.remove(labels_file)
    for i in range(len(data)):
        example = data.iloc[i]
        text = example.text
        with open(os.path.join(text_folder, "article" + str(example.id) + ".txt"), "w") as f:
            f.write(text)
        with open(labels_file, "a+") as f:
            for label in example.label:
                spl = str(label[0])
                spr = str(label[1])
                label = label[2].replace(", ", ",").replace(" / ", "/").replace(" ", "_")
                if "WT" not in label:
                    if not save_labels:
                        label = "?"
                    f.write("\t".join([str(example.id), label, spl, spr]) + "\n")

In [53]:
create_set(train, "datasets/train-articles", "datasets/train-task2-TC.labels", save_labels=True)
create_set(test, "datasets/dev-articles", "datasets/dev-task-TC-template_labeled.out", save_labels=True)
create_set(test, "datasets/dev-articles", "datasets/dev-task-TC-template.out", save_labels=False)

In [46]:
from collections import Counter

train_labels = pd.read_csv("datasets/train-task2-TC.labels", sep="\t", header=None)[1].values
dev_labels = pd.read_csv("datasets/dev-task-TC-template_labeled.out", sep="\t", header=None)[1].values
train_labels_c = Counter(train_labels)
dev_labels_c = Counter(dev_labels)
for el in train_labels_c:
    print(el, round(train_labels_c[el] / len(train_labels), 3), round(dev_labels_c.get(el, 0) / len(dev_labels), 3))

Strawman 0.017 0.019
Appeal_to_authority 0.101 0.083
Simplified_Interpretation 0.03 0.006
Loaded_language 0.076 0.109
“you_should” 0.021 0.032
Hate_speech,slang,name_calling 0.098 0.122
Sensational_and/or_provocative_headings 0.024 0.019
Causal_Simplification 0.048 0.038
False_Dilemma 0.017 0.013
Conversation_Killer 0.004 0.0
Exaggeration/Minimization 0.026 0.022
Casting_Doubt 0.085 0.103
Consequential_Simplification 0.009 0.0
Appeal_to_Hypocrisy 0.045 0.051
Statistical_deception 0.036 0.026
Appeal_to_popularity 0.008 0.016
Distraction_by_scapegoat 0.027 0.016
Appeal_to_fear/prejudice 0.034 0.058
Negative/Positive_concepts 0.045 0.026
Obfuscation,vagueness,obscurantism 0.013 0.019
Labelling 0.081 0.09
Appeal_to_Time 0.005 0.013
Substitution_of_an_idea 0.028 0.013
Guilt_by_Association 0.009 0.003
Appeal_to_values 0.029 0.022
Stereotypes 0.008 0.019
Whataboutism 0.004 0.0
Flag_waving 0.017 0.01
Rumours 0.022 0.016
Repetition 0.012 0.003
Red_Herring 0.004 0.006
Slogan 0.006 0.01
“I_am_lik

In [5]:
import numpy as np
import copy
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix


labelsn = ['Negative/Positive_concepts',
 '(PT)_Call',
 'Slogan',
 'Obfuscation,vagueness,obscurantism',
 'Consequential_Simplification',
 'Greenwashing',
 'Causal_Simplification',
 'Appeal_to_Hypocrisy',
 'Appeal_to_values',
 'Rumours',
 'Strawman',
 'Whataboutism',
 'Hate_speech,slang,name_calling',
 'Casting_Doubt',
 'Labelling',
 'Substitution_of_an_idea',
 'Statistical_deception',
 'Bluewashing',
 'Appeal_to_authority',
 'Guilt_by_Association',
 'Appeal_to_Time',
 'Flag_waving',
 '“you_should”',
 'Simplified_Interpretation',
 'Appeal_to_fear/prejudice',
 'Loaded_language',
 'Sensational_and/or_provocative_headings',
 '“I_am_like_you”',
 'Exaggeration/Minimization',
 'Red_Herring',
 'Appeal_to_popularity',
 'Repetition',
 'False_Dilemma',
 'Distraction_by_scapegoat',
 'Conversation_Killer',
 'Stereotypes']

justif = """
\item Appeal to authority (including references to an authority that is irrelevant to the topic being commented on, an expert without a confirmed current status),
\item Appeal to popularity,
\item Appeal to values, including references to virtue,
\item Appeal to fear / prejudice,
\item Greenwashing: justification through appeal to green politics,
\item Bluewashing: justification through participation in international humanitarian initiatives (often as UN projects, but not only),
\item Flag waving (appeal to national interests to justify ideas),
Rumours
"""

simple = """
\item Causal Simplification,
\item False Dilemma or no Choice,
\item Consequential Simplification,
\item Simplified Interpretation,
\item Stereotypes (an attempt to evoke negativity towards an alternative scenario, often based on prejudice),
\item ``I am like you'' or ``like everyone else'' (an attempt to convince the target audience to join in and take a course of action because ``everyone else is doing the same thing''): this is what all or ``positive'' nations / parties / groups do,
"""

distraction = """
\item Strawman,
\item Red Herring,
\item Whataboutism,
\item Appeal to Hypocrisy (‘Tu quoque’),
\item Distraction by scapegoat (a combination of ``strawman'' and adhominem - to assign someone to blame in order to remove criticism from another person),
\item Substitution of an idea / topic / issue,
"""

call = """
\item Slogan,
\item Conversation Killer,
\item Appeal to Time,
\item ``you should'', ``never do...'', ``you must...'',
(PT) Call
"""

manip = """
\item Loaded language,
\item Sensational and/or provocative headings, 
\item Repetition,
\item Exaggeration / Minimization, 
\item Obfuscation, vagueness, obscurantism,
\item Statistical deception (shift of e
"""

attack = """
\item Labelling,
\item Hate speech, slang, name calling: demonization, offensive epithets, metaphors, names related to a particular phenomenon / organization / country / nation / person / idea, etc., are used to discredit something / someone,
\item such negative concepts as ``authoritarianism'', ``aggression'', ``enemy'', ``imperialism'', ``terrorism'', ``militarism'', ``nationalism'', ``occupation'', ``racism'', ``totalitarianism'', ``junta'' are exploited. And, on the contrary, such positive concepts as ``brotherhood'', ``democracy'', ``friendship'', ``health'', ``quality'', ``love'', ``peace'', ``patriotism'', ``victory'', ``superiority'', ``prosperity'', ``equality'', ``freedom'', ``commonwealth'', ``happiness'', ``success'', etc.,
\item Casting Doubt,
\item Guilt by Association (Reductio ad Hitlerum).
"""

metalabels = {}
for label in labelsn:
    cur = label[1:-1].replace("_", " ").split("/")[0].split(",")[0]
    if cur in justif:
        metalabels[label] = "Justification"
    elif cur in simple:
        metalabels[label] = "Simplification"
    elif cur in distraction:
        metalabels[label] = "Distraction"
    elif cur in call:
        metalabels[label] = "Call"
    elif cur in manip:
        metalabels[label] = "Manipulation"
    elif cur in attack:
        metalabels[label] = "Attack on Reputation"
    else:
        print(label)
    

def replace_metalabels(preds):
    res = []
    for label in preds:
        res.append(metalabels[label])
    return np.array(res)

labels = pd.read_csv("datasets/dev-task-TC-template_labeled_meta.out", sep="\t", header=None)[1].values
preds = pd.read_csv("results/TC_output_dev_meta.txt", sep="\t", header=None)[1].values

#labels = replace_metalabels(labels)
#preds = replace_metalabels(preds)

scores = []
names = list(set(metalabels.values())) #labelsn
for label in names: #labelsn:
    dlabels = copy.deepcopy(labels)
    dpreds = copy.deepcopy(preds)
    dlabels[labels == label] = "1"
    dlabels[dlabels != "1"] = "0"
    dpreds[preds == label] = "1"
    dpreds[dpreds != "1"] = "0"
    dlabels = np.array(dlabels, dtype=int)
    dpreds = np.array(dpreds, dtype=int)
    scores.append(f1_score(dlabels, dpreds))

for name, score in zip(names, scores):
    if score != 0:
        print( name.replace("_", " ") + " & ", round(score, 3), r"\\")

Justification &  0.509 \\
Manipulation &  0.34 \\
Distraction &  0.328 \\
Attack on Reputation &  0.605 \\
Simplification &  0.3 \\
Call &  0.538 \\


In [3]:
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

labels = pd.read_csv("datasets/dev-task-TC-template_labeled_meta.out", sep="\t", header=None)[1].values
preds = pd.read_csv("results/TC_output_dev_meta.txt", sep="\t", header=None)[1].values
accuracy_score(labels, preds), f1_score(labels, preds, average="macro")

(0.483974358974359, 0.43687910686366677)

In [4]:
preds

array(['Attack on Reputation', 'Attack on Reputation', 'Justification',
       'Manipulation', 'Manipulation', 'Manipulation',
       'Attack on Reputation', 'Justification', 'Simplification',
       'Distraction', 'Simplification', 'Distraction', 'Simplification',
       'Distraction', 'Simplification', 'Manipulation', 'Justification',
       'Call', 'Simplification', 'Attack on Reputation', 'Distraction',
       'Attack on Reputation', 'Attack on Reputation',
       'Attack on Reputation', 'Attack on Reputation',
       'Attack on Reputation', 'Justification', 'Attack on Reputation',
       'Attack on Reputation', 'Attack on Reputation',
       'Attack on Reputation', 'Simplification', 'Distraction',
       'Attack on Reputation', 'Attack on Reputation', 'Distraction',
       'Distraction', 'Attack on Reputation', 'Attack on Reputation',
       'Attack on Reputation', 'Attack on Reputation',
       'Attack on Reputation', 'Attack on Reputation',
       'Attack on Reputation', 'Attack

In [133]:
df = pd.read_csv("datasets/dev-task-TC-template_labeled.out",sep="\t", header=None)
df[1] = replace_metalabels(df[1].values)
df.to_csv("datasets/dev-task-TC-template_labeled_meta.out", header=None,sep="\t", index=False)

## SI

In [51]:
import os

def create_set_si(data, labels_file):
    if os.path.exists(labels_file):
        os.remove(labels_file)
    for i in range(len(data)):
        example = data.iloc[i]
        text = example.text
        with open(labels_file, "a+") as f:
            for label in example.label:
                spl = str(label[0])
                spr = str(label[1])
                label = label[2].replace(", ", ",").replace(" / ", "/").replace(" ", "_")
                if "WT" not in label:
                    f.write("\t".join([str(example.id), spl, spr]) + "\n")

In [54]:
create_set_si(train, "datasets/train-task1-SI.labels")
create_set_si(test, "results/dev-task-SI.labels")

In [22]:
import pandas as pd
labels = pd.read_csv("results/dev-task-SI.labels", sep="\t", header=None)
preds = pd.read_csv("results/SI_output_dev.txt", sep="\t", header=None)
set(labels[0].values) - set(preds[0].values), set(preds[0].values) - set(labels[0].values)

({46, 51, 62, 122}, set())

In [58]:
from transformers import RobertaConfig

In [68]:
RobertaConfig().to_json_string()

'{\n  "attention_probs_dropout_prob": 0.1,\n  "bos_token_id": 0,\n  "eos_token_id": 2,\n  "gradient_checkpointing": false,\n  "hidden_act": "gelu",\n  "hidden_dropout_prob": 0.1,\n  "hidden_size": 768,\n  "initializer_range": 0.02,\n  "intermediate_size": 3072,\n  "layer_norm_eps": 1e-12,\n  "max_position_embeddings": 512,\n  "model_type": "roberta",\n  "num_attention_heads": 12,\n  "num_hidden_layers": 12,\n  "pad_token_id": 1,\n  "position_embedding_type": "absolute",\n  "transformers_version": "4.3.3",\n  "type_vocab_size": 2,\n  "use_cache": true,\n  "vocab_size": 30522\n}\n'

In [6]:
import pandas as pd
preds = pd.read_csv("results/SI_output_dev.txt", sep="\t", header=None)
preds[3] = ["?"] * len(preds)
preds = preds[[0, 3, 1, 2]]
preds.to_csv("results/SI_output_dev_TC_input.txt", index=False, sep="\t", header=None)

In [18]:
"""Negative/Positive_concepts
(PT)_Call
Slogan
Obfuscation,vagueness,obscurantism
Consequential_Simplification
Greenwashing
Causal_Simplification
Appeal_to_Hypocrisy
Appeal_to_values
Rumours
Strawman
Whataboutism
Hate_speech,slang,name_calling
Casting_Doubt
Labelling
Substitution_of_an_idea
Statistical_deception
Bluewashing
Appeal_to_authority
Guilt_by_Association
Appeal_to_Time
Flag_waving
“you_should”
Simplified_Interpretation
Appeal_to_fear/prejudice
Loaded_language
Sensational_and/or_provocative_headings
“I_am_like_you”
Exaggeration/Minimization
Red_Herring
Appeal_to_popularity
Repetition
False_Dilemma
Distraction_by_scapegoat
Conversation_Killer
Stereotypes""".split()

['Negative/Positive_concepts',
 '(PT)_Call',
 'Slogan',
 'Obfuscation,vagueness,obscurantism',
 'Consequential_Simplification',
 'Greenwashing',
 'Causal_Simplification',
 'Appeal_to_Hypocrisy',
 'Appeal_to_values',
 'Rumours',
 'Strawman',
 'Whataboutism',
 'Hate_speech,slang,name_calling',
 'Casting_Doubt',
 'Labelling',
 'Substitution_of_an_idea',
 'Statistical_deception',
 'Bluewashing',
 'Appeal_to_authority',
 'Guilt_by_Association',
 'Appeal_to_Time',
 'Flag_waving',
 '“you_should”',
 'Simplified_Interpretation',
 'Appeal_to_fear/prejudice',
 'Loaded_language',
 'Sensational_and/or_provocative_headings',
 '“I_am_like_you”',
 'Exaggeration/Minimization',
 'Red_Herring',
 'Appeal_to_popularity',
 'Repetition',
 'False_Dilemma',
 'Distraction_by_scapegoat',
 'Conversation_Killer',
 'Stereotypes']

In [58]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

def get_labels(preds):
    labels = []
    for el in preds:
        if len(el.split("\t")) > 1:
            labels.append(el.split("\t")[1].strip()[-4:])
            if el.startswith("см.\t"):
                labels.append("O")
    return labels

with open("model_checkpoints/si_roberta/test_predictions.txt", "r") as f:
    preds = f.readlines()
with open("cached_datasets/SI/test_labeled.tsv", "r") as f:
    gold = f.readlines()

ValueError: Found input variables with inconsistent numbers of samples: [20853, 20841]

In [112]:
for i in range(len(preds)):
    if gold[i][:2] != preds[i][:2]:
        print(i)

3167
3168
3350
5977
8554
8566
8567
9268
11032
11033
12458
12459
12485
12486
14142
14143
21328
21330
21331


In [120]:
len(" ".join(preds).split("."))

1102

In [111]:
preds.insert(21328, ':\tO\n')

In [114]:
f1_score(get_labels(gold), get_labels(preds), pos_label="PROP")

0.36365059486537255

In [123]:
sent_preds = []
for el in " ".join(preds).split("."):
    if "PROP" in el:
        sent_preds.append("PROP")
    else:
        sent_preds.append("O")
sent_gold = []
for el in " ".join(gold).split("."):
    if "PROP" in el:
        sent_gold.append("PROP")
    else:
        sent_gold.append("O")
f1_score(sent_gold, sent_preds, pos_label="PROP")

0.42659974905897113

In [24]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [25]:
from span_identification.dataset import *


def get_train_dev_files(articles_id, articles_content, nlp, labels_path, train_file, dev_file, split_by_ids=True, 
                     dev_size=0.3, random_state=42):
    articles_content_dict = dict(zip(articles_id, articles_content))
    articles_id, gold_spans = read_predictions_from_file(labels_path)
    span_list = list(zip(articles_id, gold_spans))
    train_data = sorted(group_spans_by_article_ids(span_list).items())
    train_ids = [example[0] for example in train_data]
    
    create_BIO_labeled(train_file, train_data, articles_content_dict, nlp)
    
    return train_ids

In [44]:
articles_content_v2 = []
for el in articles_content:
    articles_content_v2.append(el.replace("см.", "см ."))

In [45]:
articles_content, articles_id, propaganda_techniques_names = load_data("datasets/dev-articles", 
                                                                   "tools/data/russian_corpus_techniques.txt")
train_file_path = os.path.join("cached_datasets/SI/", "test_labeled.tsv")
get_train_dev_files(articles_id, articles_content_v2, nlp, "results/dev-task-SI.labels", train_file_path,
                                         "", True, 0.18, 57)

100%|██████████| 25/25 [00:02<00:00,  9.47it/s]


['103',
 '110',
 '111',
 '115',
 '12',
 '122',
 '123',
 '128',
 '13',
 '135',
 '14',
 '19',
 '26',
 '34',
 '35',
 '44',
 '46',
 '47',
 '51',
 '62',
 '66',
 '68',
 '88',
 '94',
 '99']